In [ ]:
!pip install seaborn
!pip install statsmodels==0.12.1

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
import seaborn as sns

In [ ]:
# import time series dataset
df = pd.read_csv("/content/anti_social_behaviour.csv", encoding='gbk', parse_dates=True,index_col=0)

# define the data for arima model
def result_arima_model(data,arima_order):
    data = data.astype('float32')
    total_data = data
    # let 70% of data be the training set
    # let 30% of data be the test set
    cut_point = int(len(data)*0.7)
    train,test = total_data[0:cut_point],total_data[cut_point:]
    # obtain values for the training set
    # define the list that stores all predictions 
    history = [x for x in train]
    prediction = []
    for t in range(len(test)):
        model = ARIMA(history,order=arima_order)
        model_fit = model.fit()
        y_pre = model_fit.forecast()[0]
        prediction.append(y_pre)
        # iterate prediction, append the result into history as training set
        history.append(test[t])
    # define rmse
    rmse = np.sqrt(mean_squared_error(test,prediction))
    mae = mean_absolute_error(test,prediction)
    return rmse

# define a function that finds out best fit p,d,q for us
def find_pdq(dataset,p_range,d_range,q_range):
    data_input = dataset
    best_score,best_order = np.inf,None
    for p in p_range:
        for d in d_range:
            for q in q_range:
                
                try:
                   rmse = result_arima_model(dataset,(p,d,q))
                   if rmse < best_score:
                       best_score = rmse
                       best_order = (p,d,q)
                   print("rmse:%.3f,order:%s" %(rmse,(p,d,q)))
                except:
                   continue
    print("best order is %s, socre is %.3f" %(best_order,rmse))
    return best_score,best_order
# print best p,d,q
find_pdq(df.values,[1,2,4,6,8,10],[0,1,2],[0,1,2])

In [ ]:
# test our model with p,d,q
cridam = pd.read_csv('/content/anti_social_behaviour.csv', index_col='time', parse_dates=['time'])
X = cridam.values
size = int(len(X) * 0.7)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
xs = np.array([i for i in range(len(train)+1)])
xs_test = np.array([i+len(train) for i in range(len(test))])

for t in range(len(test)):
    model = ARIMA(history, order=(8, 0, 1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(xs_test, test, color = 'green')
pyplot.plot(xs, X[0:size+1], color = 'blue')
pyplot.plot(xs_test, predictions, color='red')
pyplot.show()

In [ ]:
# model implication
cridam = pd.read_csv('/content/anti_social_behaviour.csv', index_col='time', parse_dates=['time'])
history = [x for x in cridam.values]
predictions = [cridam.values[-1]]
for t in range(12):
    model = ARIMA(history, order=(8, 0, 1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = yhat
    history.append(output)
    print('predicted=%f' % (yhat))
# load 2020 data
cridam2020 = pd.read_csv('/content/antisocial_real.csv', index_col='time', parse_dates=['time'])
month_in_datatime2020 = cridam2020.index.values
month_in_datatime1719 = df.index.values
month_1720_datatime = np.concatenate([month_in_datatime1719, month_in_datatime2020])
values1720 = np.concatenate([cridam.values, cridam2020.values], axis=0)
# plot
X = cridam.values
month_x = [i for i in range(0, len(X))]
month_2022 = [i+len(X)-1 for i in range(0, len(predictions))]

# pyplot.plot(month_in_datatime,X, color='gray', label='actual')
pyplot.plot(month_1720_datatime[-len(predictions):], predictions, color='red', marker='*',label='predicted')
pyplot.plot(month_1720_datatime,values1720, color='blue',marker='o', label='actual')
pyplot.legend()
pyplot.xticks(rotation=30)
pyplot.ylabel('frequency')
pyplot.title('antisocial_behaviour')
pyplot.show()